In [1]:
%cd /ibex/user/slimhy/PADS/code
"""
Test demo for MLP mapper.
"""

import argparse
import numpy as np

import torch
import torch.backends.cudnn as cudnn

import models.s2vs as autoencoders
import util.misc as misc

/ibex/user/slimhy/PADS/code


In [2]:
def get_args_parser():
    parser = argparse.ArgumentParser("Latent Diffusion", add_help=False)

    # Model parameters
    parser.add_argument(
        "--model",
        default="kl_d512_m512_l8_edm",
        type=str,
        metavar="MODEL",
        help="Name of model to train",
    )
    parser.add_argument(
        "--batch_size",
        default=32,
        type=int,
        help="Batch size per GPU (effective batch size is batch_size * accum_iter * # gpus",
    )
    parser.add_argument(
        "--text_model_name",
        type=str,
        help="Text model name to use",
    )
    parser.add_argument(
        "--ae",
        type=str,
        metavar="MODEL",
        help="Name of autoencoder",
    )
    parser.add_argument(
        "--ae-latent-dim",
        type=int,
        default=512*8,
        help="AE latent dimension",
    )
    parser.add_argument(
        "--ae_pth",
        required=True,
        help="Autoencoder checkpoint"
    )
    parser.add_argument(
        "--point_cloud_size",
        default=2048,
        type=int,
        help="input size"
    )
    parser.add_argument(
        "--fetch_keys",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--use_embeds",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--resume",
        default="",
        help="Resume from checkpoint"
    )
    parser.add_argument(
        "--resume_weights",
        action="store_true",
        default=False,
        help="Only resume weights, not optimizer state",
    )
    parser.add_argument(
        "--resume_full_weights",
        action="store_true",
        default=False,
        help="Resume the full model weights with the EDM wrapper",
    )

    # Dataset parameters
    parser.add_argument(
        "--device", default="cuda", help="device to use for training / testing"
    )
    parser.add_argument("--seed", default=0, type=int)
    parser.add_argument("--num_workers", default=60, type=int)
    parser.add_argument(
        "--pin_mem",
        action="store_true",
        help="Pin CPU memory in DataLoader for more efficient (sometimes) transfer to GPU.",
    )

    return parser


# Set dummy arg string to debug the parser
call_string = """--ae_pth ckpt/ae_m512.pth \
    --ae kl_d512_m512_l8 \
    --ae-latent-dim 4096 \
    --num_workers 8 \
    --batch_size 2 \
    --device cuda \
    --fetch_keys \
    --use_embeds \
    --seed 0"""

# Parse the arguments
args = get_args_parser()
args = args.parse_args(call_string.split())
args.data_path = "/ibex/project/c2273/PADS/3DCoMPaT/"

# --------------------
device = torch.device(args.device)

# Fix the seed for reproducibility
seed = args.seed + misc.get_rank()
torch.manual_seed(seed)
np.random.seed(seed)

cudnn.benchmark = True
args.fetch_keys = True
# --------------------

# Instantiate autoencoder
ae = autoencoders.__dict__[args.ae]()
ae.eval()
print("Loading autoencoder %s" % args.ae_pth)
ae.load_state_dict(torch.load(args.ae_pth, map_location="cpu")["model"])
ae = ae.to(device)

Loading autoencoder ckpt/ae_m512.pth


In [3]:
from datasets.metadata import class_to_idx 
import models.diffusion as diffusion
from util import misc
import util.s2vs as s2vs


def load_ckpt(model_type):
    args.model = model_type
    args.resume = "/ibex/user/slimhy/PADS/code/ckpt/dm/kl_d512_m512_l8.pth"

    model = diffusion.__dict__[model_type]()
    model.eval()
    
    misc.load_model(args, model)
    return model.to(device)


@torch.inference_mode()
def get_cond(class_name, n_samples=1):
    class_id = class_to_idx(class_name, dataset="shapenet")
    cond = torch.Tensor([class_id]*n_samples).long().to(device)
    return cond


def decode_latents(model, latents, grid_density=128, batch_size=128**3):
    # Decode the latents
    with torch.no_grad():
        mesh = s2vs.decode_latents(
            ae=model,
            latent=latents[0].unsqueeze(0),
            grid_density=grid_density,
            batch_size=batch_size,
        )
    return mesh

In [4]:
from datasets.metadata import class_to_hex
from datasets.latents import ShapeLatentDataset, ComposedPairedShapesLoader, PairType
import util.misc as misc


# Create your datasets
dataset_train = ShapeLatentDataset(
    args.data_path,
    class_code=class_to_hex("chair"),
    split="train",
    shuffle_parts=True,
    filter_n_ids=8
)

loader = ComposedPairedShapesLoader(
    dataset_train,
    batch_size=args.batch_size,
    pair_types_list=[PairType.NO_ROT_PAIR],
    num_workers=4,
    seed=0,
    shuffle=False,
    drop_last=True,
)

l_a, bb_a, bb_l_a, cls_label_a, meta_a = loader.get_tuple(device=device)

/home/slimhy/conda/envs/3D2VS_flexicubes/lib/python3.10/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [5]:
import torch
from models.part_queries import PQM
from models.diffusion import EDMPartQueries

# Initialize models
pqm = PQM(dim=512, heads=8, dim_head=64)
model = EDMPartQueries(part_queries_encoder=pqm, n_latents=512, channels=8, depth=24)
model = model.cuda()

# Initialize your dataloader
dataset_train = ShapeLatentDataset(
    args.data_path,
    class_code=class_to_hex("chair"),
    split="train",
    shuffle_parts=True,
    filter_n_ids=8
)

loader = ComposedPairedShapesLoader(
    dataset_train,
    batch_size=16,
    pair_types_list=[PairType.NO_ROT_PAIR],
    num_workers=4,
    seed=0,
    shuffle=False,
    drop_last=True,
)

# Get data from loader
l_a, bb_a, bb_l_a, cls_label_a, meta_a = loader.get_tuple(device="cuda")

# Debug: Print value ranges
print(f"bb_l_a min: {bb_l_a.min()}, max: {bb_l_a.max()}")
print(f"Part label embedding num_embeddings: {pqm.part_embed.part_label_embed.num_embeddings}")

bb_l_a min: -1, max: 266
Part label embedding num_embeddings: 275


In [6]:
# Create mask (use valid part labels as mask)
mask_a = (bb_l_a >= 0) & (bb_l_a < pqm.part_embed.part_label_embed.num_embeddings)
print(f"Valid labels percentage: {mask_a.float().mean()*100:.2f}%")

# Print shapes before forward pass
print(f"l_a shape: {l_a.shape}")
print(f"bb_a shape: {bb_a.shape}")
print(f"bb_l_a shape: {bb_l_a.shape}")

mask_a = (bb_l_a == -1)

# Pack data and run forward pass
samples = (l_a, bb_a, bb_l_a, mask_a)
sigma = torch.ones(l_a.shape[0], device="cuda")

try:
    out, part_embeds = model(samples=samples, sigma=sigma)
    print("Success!")
    print(f"Output shape: {out.shape}")
    print(f"Part embeds shape: {part_embeds.shape}")
except RuntimeError as e:
    print(f"Error: {e}")

Valid labels percentage: 38.02%
l_a shape: torch.Size([8, 512, 8])
bb_a shape: torch.Size([8, 24, 4, 3])
bb_l_a shape: torch.Size([8, 24])
Success!
Output shape: torch.Size([8, 512, 8])
Part embeds shape: torch.Size([8, 24, 512])
